#### Pre-process

In [19]:
## extract VCF info (overlapping vcf)
os.chdir("/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/by_sample/overlap_peak/reorder/")
result_dir = "overlap_vcf/"
os.system("mkdir -p %s" % result_dir)
ind_dict = {
            "ind1": ['brain_IP_1', 'heart_IP_1', 'liver_IP_1'],  
            "ind2": ['brain_IP_2', 'heart_IP_2', 'placenta_IP_2', 'kidney_IP_2', 'liver_IP_2'],  
            "ind3": ['brain_IP_3', 'heart_IP_3', 'liver_IP_3', 'kidney_IP_3'], 
            "ind4": ['stomach_IP_4', 'muscle_IP_4', 'lung_IP_4', 'kidney_IP_4', 'placenta_IP_4'], 
            "ind5": ['stomach_IP_5', 'muscle_IP_5', 'lung_IP_5'], 
            "ind6": ['placenta_IP_6'],
            "ind7": ['Adipose-1-1', 'Adrenal_gland-1-1', 'Aorta-1-1', 'Heart-1-1', 'Skin-1-1', 'Spleen-1-1'], 
            "ind8": ['Lung-2-1', 'Lung-2-4', 'Spleen-2-1', 'Tongue-2-1', 'Urinary_bladder-2-1'], 
            "ind9": ['Appendix-3-2', 'Colon-3-2', 'Esophagus-3-2', 'Muscle-3-2', 'Spleen-3-2'],
            "ind10": ['Aorta-4-2', 'Esophagus-4-2', 'Heart-4-2', 'Jejunum-4-2', 'Liver-4-2','Lung-4-2','Lung-4-4','Prostate-4-2','Rectum-4-2','Skin-4-2','Stomach-4-2','Testis-4-2','Thyroid_gland-4-2','Urinary_bladder-4-2'],
            "ind11": ['Appendix-5-3', 'Brainstem-5-3', 'Cerebellum-5-3', 'Cerebrum-5-3', 'Duodenum-5-3', 'Hypothalamus-5-3','Jejunum-5-3','Muscle-5-3','Rectum-5-3','Stomach-5-3','Thyroid_gland-5-3','Trachea-5-3','Urinary_bladder-5-3'], 
            "ind12": ['Cerebrum-6-3'],
            "ind13": ['Cerebellum-7-4'],
            "ind14": ['FrontalCortex_1','Cerebellum_1','Heart_1','Liver_1','Lung_1','Kidney_1','Spleen_1'],
            "ind15": ['FrontalCortex_2','Cerebellum_2','Heart_2','Liver_2','Kidney_2','Spleen_2','Muscle_1'],
            "ind16": ['FrontalCortex_3','Cerebellum_3','Heart_3','Liver_3','Lung_2','Muscle_2'],
            "ind17": ['Muscle_3'],
            "ind18": ['Lung_3','Kidney_3','Spleen_3']
           }
vcf_dir = "/Charles/project/ASm6A/by_ind/SNP_calling/vcf_dir/rm_repeat_editing/overlap_dbsnp/"
vcf_list = glob.glob("%s/ind*_gatk.vcf" % vcf_dir)
with open("overlap.sh", 'w') as fw:
    for vcf in vcf_list:
        p = os.path.basename(vcf).split("_gatk.vcf")[0]
        sample_list = ind_dict[p]
        for sample in sample_list:
            bed = "%s.bed" % sample
            res = os.path.join(result_dir, "%s.txt" % sample)
#             os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (bed, vcf, res))
            fw.write("bedtools intersect -a %s -b %s -wa -wb > %s\n" % (bed, vcf, res))

#### Select candidate SNPs

In [1]:
## tag SNP appeared in multiple samples
info_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/by_sample/overlap_peak/reorder/overlap_vcf/"

def group_snp(df):
    joint_snp = "@".join(df['SNP'].tolist())
    df = pd.DataFrame(np.array([[df.iloc[0,0], df.iloc[0,1], df.iloc[0,2], joint_snp]]))
    return df

def reform_file(txt): # ASm6A + Peak + SNP
    sample = os.path.basename(txt).split(".txt")[0]
    df = pd.read_table(txt, header=None)
    df['term'] = df.iloc[:,5] + ":" + df.iloc[:,7].astype(str)
    df['AR'] = df.iloc[:,8].str.split(";").str[1]
    df['peak'] = df.iloc[:,0] + ":" + df.iloc[:,1].astype(str) + "-" + df.iloc[:,2].astype(str)
    df['SNP'] = df.iloc[:,11] + ":" + df.iloc[:,12].astype(str) + "-" + df.iloc[:,20].str.split(":").str[0]
    df = df[['term', 'AR', 'peak', 'SNP']]
    ## merge SNP into one row for each peak
    df = df.groupby(['term', 'AR', 'peak'], as_index=False).apply(group_snp)
    df.columns = ['term', 'AR', 'peak', 'SNP']
    df['value'] = sample + ";" + df['AR'] + ";" + df['peak'] + ";" + df['SNP']
    return df[['term', 'value']]

bed_list, df_list, term_dict = glob.glob("%s/*.txt" % info_dir), [], {}
for bed in bed_list:
    df_list.append(reform_file(bed))
df = pd.concat(df_list)

def reform_dict(in_dict):
    out_dict = {}
    for term, value_list in in_dict.items():
        sample_list = [value.split(";")[0] for value in value_list]
        ar_list     = [value.split(";")[1] for value in value_list]
        peak_list   = [value.split(";")[2] for value in value_list]
        snp_list    = [value.split(";")[3] for value in value_list]
        out_dict[term] = {"sample": sample_list, "AR": ar_list, "peak": peak_list, "SNP": snp_list}
    return out_dict

for i, values in df.iterrows():
    term_dict[values['term']] = term_dict.get(values['term'], []) + [values['value']]
term_dict = reform_dict(term_dict)
print(term_dict['chr1:6695123'])

####  Candidate SNPs
#### Extract SNP list contained in m6A peaks which contain ASm6As
#### m6A peaks: the regions within which to search for candidate functional SNPs.

In [1]:
os.chdir("/Charles/project/ASm6A/ASm6A/withPeak/")
result_dir = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/candidateSNP/"
os.system("mkdir -p %s" % result_dir)
vcf_dir = "/Charles/project/ASm6A/corrected_vcf/vcf/rename/"

bed_list = glob.glob("*.bed")
for bed in bed_list:
    res = os.path.join(result_dir, os.path.basename(bed))
    df = pd.read_table(bed, header=None)
    df = df.iloc[:,6:-1]
    df.to_csv(res, sep="\t", header=False, index=False)
    ### overlap vcf
    p = os.path.basename(bed).split(".bed")[0]
    vcf = os.path.join(vcf_dir, "%s_gatk.vcf"%p)
    tag_vcf = os.path.join(result_dir, bed.replace(".bed", ".vcf"))
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (vcf, res, tag_vcf))
    os.remove(res)

In [18]:
#### reformat candidate SNP and peak
os.chdir("/Charles/project/ASm6A/ASm6A/functional_ASm6A/candidateSNP/")
vcf_list = glob.glob("*.vcf")
result_dir = "reformat/"
os.system("mkdir -p %s" % result_dir)

for vcf in vcf_list:
    sample = vcf.split(".vcf")[0]
    df = pd.read_table(vcf, sep="\t", header=None)
    df['m6Apeak'] = df.iloc[:,-4] + ":" + df.iloc[:,-3].astype(str) + "-" + df.iloc[:,-2].astype(str)
    df['candidateSNP'] = df.iloc[:,0]+":"+df.iloc[:,1].astype(str)
    df['SNP_info'] = df.iloc[:,2]+":"+df.iloc[:,3]+":"+df.iloc[:,4]
    df['genotype'] = df.iloc[:,9].astype(str).str.split(":").str[0]
    df['alleleReads'] = df.iloc[:,9].astype(str).str.split(":").str[1]
    df['sample'] = sample
    result_file = os.path.join(result_dir, vcf.replace(".vcf", ".txt"))
    df = df[['candidateSNP','genotype','SNP_info','alleleReads','m6Apeak','sample']].drop_duplicates()
    df.to_csv(result_file,sep="\t",index=False)

#### Select tagSNP

In [20]:
os.chdir("/Charles/project/ASm6A/ASm6A/withPeak/")
bed_list = glob.glob("*.bed")
print(len(bed_list))
result_dir = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/tagSNP/"
os.system("mkdir -p %s" % result_dir)

for bed in bed_list:
    sample = bed.split(".bed")[0]
    df = pd.read_table(bed, header=None)
    df['tagSNP'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
    df['allelicRatio'] = df.iloc[:,3].str.split(";").str[1]
    df['mark'] = df.iloc[:,3].str.split(";").str[2]
    df['m6Apeak'] = df.iloc[:,6] + ":" + df.iloc[:,7].astype(str) + "-" + df.iloc[:,8].astype(str)
    df['sample'] = sample
    df = df[['tagSNP', 'allelicRatio', 'mark', 'm6Apeak', 'sample']].drop_duplicates()
    result_file = os.path.join(result_dir, bed.replace(".bed", ".txt"))
    df.to_csv(result_file, sep="\t", index=False)

90


#### Generate a unit m6A peak pool (90 samples)

In [2]:
import random
##############################################################################
############## !!!!!!!!!!!!!  skip !!!!!!!!!!!!!!!!!!#########################
#### define unified peak region
#### for each tagSNP, combine the m6A peaks in the corresponding individuals
##############################################################################
## step 1: pick out tagSNP (total 90 samples)
tagSNP = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/tagSNP/combined/totalSample_tagSNP.txt"
df_tagSNP = pd.read_table(tagSNP, header=None)
df_tagSNP.columns = ["tagSNP", "AR", "mark", "peak", "sample"]
# tagSNP_list = list(set(df_tagSNP.iloc[:,0].tolist()))
# print(df_tagSNP.head())
# count_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/rename/"
## step 2: select only one cooresponding m6A peak randomly
## potential problems (one m6A peak contains two tagSNPs) --- Not yet considered 
tagSNP_dict = {}
for i, values in df_tagSNP.iterrows():
    tagSNP_dict[values['tagSNP']] = tagSNP_dict.get(values['tagSNP'], []) + [values['peak']]
peak_dict = {}
for tagSNP, peak_list in tagSNP_dict.items():
    peak_dict[tagSNP] = random.sample(peak_list, 1)[0]
# print(peak_dict)
## step 3: pick out candidate SNPs according to the m6A peak
candSNP="/Charles/project/ASm6A/ASm6A/functional_ASm6A/candidateSNP/reformat/combined/totalSample_candidateSNP.txt"
df_cand = pd.read_table(candSNP)
ar = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/rename/combined/totalSample_SNP.txt"
df_ar = pd.read_table(ar)

def pick_out_candidateSNP(m6A_peak, tagSNP):
    df = df_cand[df_cand['m6Apeak'] == m6A_peak]
    df['SNP'] = tagSNP ### as a condition for filtering
#     candSNP_samples = df['sample'].tolist()
    ### two conditions need to be met
    ### 1. tagSNP position; 2. candidate SNP sample
    df_merge = df.merge(df_ar, on=['sample', 'SNP'])
    return df_merge

df_list = []
for tagSNP, peak in peak_dict.items():
    df = pick_out_candidateSNP(peak, tagSNP)
#     df['tagSNP'] = df['SNP']
#     del df['SNP']
#     print(df.head())
    df_list.append(df)
df = pd.concat(df_list)
df.to_csv("/Charles/project/ASm6A/ASm6A/functional_ASm6A/matching_data/test_for_functionASm6A.txt",sep="\t",index=False)

#### Matching candidateSNPs and tagSNPs across individuals

In [ ]:
tagSNP_dir = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/tagSNP/"
candidateSNP_dir = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/candidateSNP/reformat/"
result_dir = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/matching_data/"
os.system("mkdir -p %s" % result_dir)
####
tagSNP_list = glob.glob("%s/*.txt" % tagSNP_dir)
for tagSNP in tagSNP_list:
    candidateSNP = os.path.joino(candidateSNP_dir, os.path.basename(tagSNP))
    

#### Generate genotype and allele depth dict across 90 samples

In [1]:
info_dir = "/Charles/project/ASm6A/corrected_vcf/vcf/rename/extract_info/"
vcf_list = glob.glob("%s/*.vcf" % info_dir)
info_dict = {} # 
for vcf in vcf_list:
    sample_n = os.path.basename(vcf).split("_gatk.vcf")[0]
    info_dict[sample_n] = {"heter": {}, "homo": {}}
for vcf in vcf_list:   
    sample_n = os.path.basename(vcf).split("_gatk.vcf")[0]
    df = pd.read_table(vcf)
    #### filter based on read depth
    df['refCount'] = df['GEN[*].AD'].str.split(",").str[0].astype(int)
    df['altCount'] = df['GEN[*].AD'].str.split(",").str[1].astype(int)
    df = df[(df['refCount']>=2) & (df['altCount']>=2) & (df['refCount']+df['altCount'] >=10)]
    ####
    df['term'] = df['CHROM'] + ":" + df['POS'].astype(str)
    df_heter = df[df['GEN[*].GT'] == "0/1"]
    df_homo = df[df['GEN[*].GT'] == "1/1"]
    info_dict[sample_n]['heter'] = dict(zip(df_heter['term'], df_heter['GEN[*].AD']))
    info_dict[sample_n]['homo'] = dict(zip(df_homo['term'], df_homo['GEN[*].AD']))
#### 
print(info_dict['liver_IP_3']['heter']['chr1:14574']) # 24,8
print(info_dict['liver_IP_3']['homo']['chr1:565870']) # 3,5597

24,8
3,5597


#### query tagSNP acorss 90 samples

In [2]:
tagSNP = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/tagSNP/tagSNP.txt"
df = pd.read_table(tagSNP, header=None)
df['term'] = df.iloc[:,0] + ":" + df.iloc[:,1].astype(str)
tagSNP_list = list(set(df['term'].tolist()))

vcf_dir = "/Charles/project/ASm6A/corrected_vcf/vcf/rename/extract_info/"
sample_list = [os.path.basename(x).split("_gatk.vcf")[0] for x in glob.glob("%s/*_gatk.vcf" % vcf_dir)]
snp_dict = {}
for tagSNP in tagSNP_list:
    snp_dict[tagSNP] = {"sample": [], "AD": [], "type": []}
    for sample in sample_list:
        if tagSNP in info_dict[sample]['heter']:
            snp_dict[tagSNP]['sample'].append(sample)
            snp_dict[tagSNP]['AD'].append(info_dict[sample]['heter'][tagSNP])
            snp_dict[tagSNP]['type'].append("heter")
        if tagSNP in info_dict[sample]['homo']:
            snp_dict[tagSNP]['sample'].append(sample)
            snp_dict[tagSNP]['AD'].append(info_dict[sample]['homo'][tagSNP])
            snp_dict[tagSNP]['type'].append("homo")
###
print(snp_dict['chr7:128292658'])

{'sample': ['placenta_IP_6', 'FrontalCortex_3', 'stomach_IP_4', 'Spleen_1', 'Spleen_3', 'placenta_IP_4', 'Heart_1', 'kidney_IP_4', 'lung_IP_4', 'muscle_IP_4'], 'AD': ['30,27', '27,6', '47,29', '23,7', '29,8', '32,18', '10,3', '25,9', '25,20', '28,16'], 'type': ['heter', 'heter', 'heter', 'heter', 'heter', 'heter', 'heter', 'heter', 'heter', 'heter']}


#### Calculate concordance score
##### The read count comes  from  AD tag contained in vcf

In [3]:
#### concordance between genotype of candiate SNP
#### The order of ref,alt are the same.  
from collections import Counter

def calculate_heter(refCount, altCount):
    ar = int(refCount)/(int(refCount)+int(altCount))
    d = abs(0.5-ar)
    s = (d*d)/(0.5*0.5)
    return s
    
def calculate_homo(refCount, altCount):
    return 1-calculate_heter(refCount, altCount)

score_dict = {}
for snp, i_dict in snp_dict.items():
    ad_list, type_list, score_list = i_dict['AD'], i_dict['type'], []
    for i in range(len(ad_list)):
        ref_count, alt_count = ad_list[i].split(",")
        if type_list[i] == "heter":
            score_list.append(calculate_heter(ref_count, alt_count))
        elif type_list[i] == "homo":
            score_list.append(calculate_homo(ref_count, alt_count))
    assert len(ad_list) == len(type_list) == len(score_list)
    score_dict[snp] = score_list
print(score_dict['chr7:128292658'])

[0.0027700831024930687, 0.40495867768595056, 0.0560941828254848, 0.28444444444444456, 0.32213294375456536, 0.07840000000000001, 0.28994082840236696, 0.22145328719723192, 0.01234567901234569, 0.07438016528925619]


#### hypothesis test (高斯混合模型)

In [3]:
import math
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from sklearn import mixture
from sklearn.mixture import GaussianMixture
%run smartFunctions.ipynb import distributeBins
%run GMM_select_Peak.ipynb import calc_causalSNP

for snp, si_list in score_dict.items():
#     si_list = sub_dict['conc_score']
    if len(si_list) >= 10:
        calc_causalSNP(snp, si_list)

In [24]:
#### parse results
in_file = "/Charles/project/ASm6A/ASm6A/functional_ASm6A/GMM_results.txt"
df = pd.read_table(in_file, header=None, sep=" ")
df.columns = ["snv", "peakX", "zScore", "Pvalue", "nComp", "Mean", "Stdevs", "pns", "ratio"]
df['peakX'] = df['peakX'].astype(float)
df_sub = df[df['peakX'] > 0.5]
# print(len(df_sub))

,snv,peakX,zScore,Pvalue,nComp,Mean,Stdevs,pns,ratio
0,chr19:44418824,0.169550,-inf;inf|-3863474004743760.0;1.892313798241841...,0.0;0.0|0.0;0.0|0.025275351524042273;0.0|NA;NA,4,0.0744|0.1696|0.017|0.1289,0.0|0.0|0.0076|0.0151,2|2|5|3,0.2222|0.2222|0.5556|0.3333
1,chr15:31665443,0.014400,-1.2000569850689293;77.96739121638797|NA;NA|NA...,0.2301172097886507;0.0|NA;NA|NA;NA|NA;NA,4,0.0152|0.2925|0.5463|0.1227,0.0126|0.0132|0.0|0.0457,18|3|1|7,2.0|0.3333|0.1111|0.7778
2,chr11:35641844,0.001189,NA;NA|-1.0320091168319419;466.2223627495159|NA...,NA;NA|0.30206784664690856;0.0|NA;NA|0.00105216...,4,0.25|0.0022|0.16|0.0442,0.0|0.0021|0.0|0.0135,1|5|1|3,0.1111|0.5556|0.1111|0.3333
3,chr9:129266187,0.111111,NA;NA|NA;NA|-3140368188905484.5;2.512294551124...,NA;NA|NA;NA|0.0;0.0|0.32428291549358135;0.0,4,0.0574|0.16|0.1111|0.0114,0.0118|0.0|0.0|0.0116,4|1|2|6,0.4444|0.1111|0.2222|0.6667
4,chr3:195456837,0.169550,-1.4565939836530997;7.133758837431445,0.14522848941276206;9.76643876869293e-13,1,0.1696,0.1164,12,1.3333


In [ ]:
def select_common_SNPs(snp_list):
    snp_array = []
    for i_sample in snp_list:
        i_list = i_sample.split("@")
        snp_array.append(i_list)
    sample_num = len(snp_array)
    total_list, result_list = [], []
    for i_list in snp_array:
        total_list += [x.split("-")[0] for x in i_list] ### only keep position info
    count = Counter(total_list)
    for snp, i_count in count.items():
        if i_count == sample_num:
            result_list.append(snp)
    return snp_array, result_list

def cal_concor_for_eachSNP(snp_array, comm_list):
    index_list, geno_list = [], []
    for i in range(len(snp_array)):
        snp_line = "@".join(snp_array[i])
        if snp in snp_line:
            index_list.append(i)
            genotype = snp_line.split("%s-"%snp)[1].split("@")[0]
            geno_list.append(genotype)
    raw_list = [ar_list[i] for i in index_list]
    assert len(raw_list) == len(geno_list)
    score_list = []
    for i in range(len(raw_list)):
        if len(set(geno_list[i].split("/"))) > 1:
            score_list.append(calculate_heter(raw_list[i]))
        else:
            score_list.append(calculate_homo(raw_list[i]))
    return score_list, geno_list

score_dict = {}
for ASm6A, info_dict in term_dict.items():
    snp_list, ar_list, sample_list = info_dict['SNP'], info_dict['AR'], info_dict['sample']
    snp_array, comm_list = select_common_SNPs(snp_list)
    for snp in comm_list:
        score_dict[snp] = {"conc_score": [], "genotype": []}
        score_list, geno_list = cal_concor_for_eachSNP(snp_array, comm_list)
        score_dict[snp]['conc_score'] = score_list
        score_dict[snp]['genotype'] = geno_list
#
score_dict['chr10:101152191']